Practica 1. Seminario de Ciencia de Datos

Calidad de Datos

Spotify te contrató como consultor para un proyecto el cual esta a cargo de la área de Data
Quality, el problema de la calidad de datos de esta base proviene principalmente de los
artístas que son los que ingresan los datos de sus canciones, por lo cual tenemos una gran
oportunidad para poder limpiar, estandarizar y mejorar la información para tener una mejor
toma de decisiones.

1. Con los datos proporcionados realiza los siguientes puntos:

In [88]:
'''(a) Realiza el etiquetado de las variables de acuerdo a su tipo de variable'''

#Cargamos librerias y vemos los tipos de datos
import pandas as pd 
spotifydata = pd.read_excel('spotify_practica.xlsx')
print(spotifydata.dtypes)

#Etiquetamos variables
d_feats = ['duration_ms', 'mode', 'zip Code']
c_feats = ['acousticness', 'danceability', 'energy', 'explicit', 'instrumentalness', 'key', 'liveness', 'loudness', 'popularity', 'speechiness', 'tempo', 'valence']
s_feats = ['artists', 'id', 'name', 'genero']
t_feats = ['release_date']

d_feats_new = ["d_" + x for x in d_feats]
c_feats_new = ["c_" + x for x in c_feats]
s_feats_new = ["s_" + x for x in s_feats]
t_feats_new = ["t_" + x for x in t_feats]

spotifydata.rename(columns = dict(zip(c_feats, c_feats_new)), inplace = True)
spotifydata.rename(columns = dict(zip(d_feats, d_feats_new)), inplace = True)
spotifydata.rename(columns = dict(zip(t_feats, t_feats_new)), inplace = True)
spotifydata.rename(columns = dict(zip(s_feats, s_feats_new)), inplace = True)

acousticness        float64
artists              object
danceability        float64
duration_ms           int64
energy              float64
explicit            float64
id                   object
instrumentalness    float64
key                 float64
liveness            float64
loudness            float64
mode                  int64
name                 object
popularity          float64
release_date         object
speechiness         float64
tempo               float64
valence             float64
zip Code             object
genero               object
dtype: object


In [89]:
'''(b) Imprime cuántos duplicados hay en la base de datos y elíminalos'''

print(spotifydata.shape)
antes = spotifydata.shape[0]
spotifydata.drop_duplicates(inplace=True)
spotifydata.reset_index(drop=True,inplace=True)
print(spotifydata.shape)
print('Eliminamos', antes-spotifydata.shape[0], 'registros duplicados')

(170349, 20)
(169985, 20)
Eliminamos 364 registros duplicados


In [90]:
'''(c) Realiza la completitud de las variables'''

#Como vimos en clase, voy a crear un dataframe auxiliar para ver la completitud de los datos
completitudDat = pd.DataFrame(spotifydata.isnull().sum())
completitudDat.reset_index(inplace=True)
completitudDat.rename(columns={'index':'columna', 0:'nulos_total'}, inplace=True)
completitudDat['completitud'] = ((1 - completitudDat['nulos_total'] / spotifydata.shape[0]) * 100).round(2)
completitudDat = completitudDat.sort_values(by='completitud', ascending=True)
print(completitudDat)


               columna  nulos_total  completitud
5           c_explicit        62905        62.99
8                c_key        35759        78.96
13        c_popularity         3501        97.94
19            s_genero         1698        99.00
3        d_duration_ms            0       100.00
4             c_energy            0       100.00
2       c_danceability            0       100.00
0       c_acousticness            0       100.00
7   c_instrumentalness            0       100.00
6                 s_id            0       100.00
10          c_loudness            0       100.00
9           c_liveness            0       100.00
11              d_mode            0       100.00
12              s_name            0       100.00
14      t_release_date            0       100.00
1            s_artists            0       100.00
15       c_speechiness            0       100.00
16             c_tempo            0       100.00
17           c_valence            0       100.00
18          d_zip Co

In [91]:
'''(d) Elimina las variables que tengan el 20% o más de valores ausentes'''

print(spotifydata.shape)
spotifydata = spotifydata.drop(columns=['c_explicit', 'c_key'])
print(spotifydata.shape)
#Se eliminaron exitosamente las dos columnas

(169985, 20)
(169985, 18)


In [92]:
'''(e)¿Cuántos de los registros en la variable “zip Code” son valores no válidos?, es decir, contienen letras'''

num = spotifydata['d_zip Code'].str.contains(r"[A-Za-z]", na=False).sum()
print('Hay',num,'codigos postales con letras')

Hay 4341 codigos postales con letras


In [93]:
'''(f) Elimina los registros que no tengas un “zip Code” válidos.'''

spotifydata = spotifydata[~spotifydata['d_zip Code'].str.contains(r"[A-Za-z]", na=False)]
print('Nos quedamos con', spotifydata.shape[0],'registros')


Nos quedamos con 165644 registros


In [94]:
'''(g) ¿Cuántos de los registros en la variable “genero” son valores no validos?, es decir, contienen letras'''

num = spotifydata['s_genero'].str.contains(r"[A-Za-z]", na=False).sum()
print('Hay',num,'registros de genero con letras')

Hay 8 registros de genero con letras


In [95]:
'''(h) Elimina los registros que no tengas un “genero” válido, es decir contenga letras en los valores'''

spotifydata = spotifydata[~spotifydata['s_genero'].str.contains(r"[A-Za-z]", na=False)]
print('Nos quedamos con', spotifydata.shape[0],'registros')


Nos quedamos con 165636 registros


In [96]:
'''(i) Haz una limpieza de la variable “name”, elimina caracteres especiales y todo debe estar en minúsculas'''

#Creamos una funcion que elimine caracteres especiales
import unicodedata
import re
def clean_text(text, pattern="[^a-zA-Z0-9 ]"):
    
    cleaned_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore')
    cleaned_text = re.sub(pattern, " ", cleaned_text.decode("utf-8"), flags=re.UNICODE)
    cleaned_text = u' '.join(cleaned_text.lower().strip().split())
    return cleaned_text

#Limpiamos aplicando la funcion
spotifydata['s_name'] = spotifydata['s_name'].astype(str) #Agregamos esto porque parece que habia algunos registros que no eran string
spotifydata['s_name'] = spotifydata['s_name'].map(lambda x: clean_text(x.lower()).strip().replace('\n',''))
print(spotifydata['s_name'].head())

24                                   crypts of eternity
25                                mi sombra en la pared
26                                          knocked out
27                                              zumbale
28    walkin blues acoustic live at mtv unplugged br...
Name: s_name, dtype: object


In [97]:
'''(j) De la variable “artista” selecciona solo al primer artista que aparezca en la lista además de eliminar caracteres especiales e imprime los primeros 5 artistas que
aparecen en tu dataframe hasta este momento.'''

print(spotifydata['s_artists'].iloc[0])

#Limpiamos aplicando la funcion
spotifydata['s_artists'] = spotifydata['s_artists'].map(lambda x: clean_text(x.lower()).strip().replace('\n',''))
print(spotifydata['s_artists'].head())

['Slayer']
24               slayer
25    miguel mateos zas
26          paula abdul
27           henry fiol
28         eric clapton
Name: s_artists, dtype: object


In [101]:
'''(k)Normaliza la variable “genero” de tal forma que obtengas solo 10 categorías'''

spotifydata.loc[spotifydata['s_genero'].isin(['49.0', '48.0', '12.0', '47.0', '20.0', '50.0']), 's_genero'] = 'Otros'
print(len(spotifydata['s_genero'].unique()))

10


In [ ]:
'''(l) Añade las siguientes columnas a tu table base: zip, lat, lng, city, state_name utilizando la tabla zips_practica'''

